# Python RSA Implementation of Demonstrative Pronouns
### Using the RSA Python model from Frank and Goodman (2012) and Lund et al. (2019) to model semantic extension in English demonstrative pronouns

In [1]:
import numpy as np
import pandas as pd

Setting the reference areas, utterances, and probabilities for each in the model. For the time being, the probabilities are uniform for both locations, though this may change by context.

In [39]:
# Set the location possibilities for reference
locations = ["proximal", "distal"]

In [3]:
# Set the possible pronouns in the utterance
utterances = ["this", "that"]

In [5]:
# Set the probabilities for the locations as a uniform probability for reference in each state
# This may change as languages and contexts have different understandings of the field that refers to proximal and field that refers to distal
def uniform_probs():
    return pd.Series(np.ones(len(locations))/len(locations), index=locations)

In [6]:
uniform_probs()

proximal    0.5
distal      0.5
dtype: float64

### Costs and Prior
To accurately model the diachronic semantic change, we can change the costs for each demonstrative. This is backed by experimental data collected in Monks & Davidson (2021) such that the proximal is more marked, and therefore more "costly" than the distal. 

The utterance prior is only used to show how the costs influence the prior, but is not implemented later on. 

Alpha is actor optimality.

In [102]:
costs = {"this": 2.1877, 
        "that": 2}

def cost(utterance):
    return costs[utterance]

alpha = 1

In [103]:
# Only used as an illustration, not in the model
def utterance_prior():
    probs = list(map(lambda x:np.exp(-costs[x]), utterances))
    return pd.Series(probs / np.sum(probs), index = utterances)

In [104]:
utterance_prior()

this    0.453212
that    0.546788
dtype: float64

### Meaning Function

For a given utterance, returns true or false and the location of the item as marked in the [+prox] feature for the proximal

In [105]:
# Meaning function reflects that the proximal is marked for location and the distal is unmarked for location
def meaning(utterance, location):
    if utterance == 'this':
        if location == 'proximal':
            return True
    elif utterance == 'that':
        if location in ['proximal', 'distal']:
            return True
    else:
        return False

### Literal Listener

Returns a probability distribution over locations for a given utterance.

In [106]:
# Returns a distribution over locations
def literal_listener(utterance):
    truth_values = np.array(list(map(lambda x:meaning(utterance, x), locations)))
    return truth_values * uniform_probs() / pd.Series.sum(truth_values * uniform_probs())

In [107]:
def L1_wrapper():
    return (pd.DataFrame({k:literal_listener(k) for k in utterances})).fillna(0)

In [108]:
L1_wrapper()

,this,that
proximal,1.0,0.5
distal,0.0,0.5


### Pragmatic Speaker

Returns a distribution over utterances for a given location

In [109]:
def utility(L1_probs):
    return L1_probs.apply(lambda x:np.exp(alpha * np.log(x) - cost(x.name)))

In [110]:
def speaker():
    L1 = L1_wrapper()
    return utility(L1).apply(lambda x:x / np.sum(x), axis = 1)

In [111]:
speaker()

/Users/tessmonks/opt/anaconda3/envs/cs187/lib/python3.8/site-packages/pandas/core/arraylike.py:364: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


,this,that
proximal,0.623739,0.376261
distal,0.000000,1.000000


### Pragmatic Listener

Returns a distribution over objects for a given utterance.

In [112]:
def pragmatic_listener():
    S1 = speaker()
    return S1.transpose().apply(lambda x: x * uniform_probs() / np.sum(x * uniform_probs()), axis = 1)

In [113]:
pragmatic_listener()

/Users/tessmonks/opt/anaconda3/envs/cs187/lib/python3.8/site-packages/pandas/core/arraylike.py:364: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


,proximal,distal
this,1.000000,0.000000
that,0.273394,0.726606
